# 1. Etração dos dados

### imports

In [513]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import re

### Criando conexão os bancos de dados (transacional e DW)

In [514]:
DB_HOST_ORIGEM = "localhost"
DB_NAME_ORIGEM = "crm_transacional_db"
DB_USER_ORIGEM = "crm_user"
DB_PASSWORD_ORIGEM = "crm_password"
DB_PORT_ORIGEM = "5432"

conexao_origem_str = (
    f"postgresql://{DB_USER_ORIGEM}:{DB_PASSWORD_ORIGEM}@"
    f"{DB_HOST_ORIGEM}:{DB_PORT_ORIGEM}/{DB_NAME_ORIGEM}"
)
engine_origem = create_engine(conexao_origem_str)


DB_HOST_DW = "localhost"
DB_NAME_DW = "dw_db"
DB_USER_DW = "dw_user"
DB_PASSWORD_DW = "dw_password"
DB_PORT_DW = "5433"

conexao_dw_str = (
    f"postgresql://{DB_USER_DW}:{DB_PASSWORD_DW}@"
    f"{DB_HOST_DW}:{DB_PORT_DW}/{DB_NAME_DW}"
)
engine_dw = create_engine(conexao_dw_str)

### Consumindo dados

In [515]:
query_clientes = "SELECT * FROM CLIENTE"
query_categoria_clientes = "SELECT * FROM CATEGORIA_CLIENTE"
query_categoria_produto = "SELECT * FROM CATEGORIA_PRODUTO"
query_fornecedores = "SELECT * FROM FORNECEDORES"
query_item_vendas = "SELECT * FROM ITEM_VENDAS"
query_localidade = "SELECT * FROM LOCALIDADE"
query_lojas = "SELECT * FROM LOJAS"
query_produtos = "SELECT * FROM PRODUTO"
query_produto_fornecedor = "SELECT * FROM PRODUTO_FORNECEDOR"
query_promocoes = "SELECT * FROM PROMOCOES"
query_vendas = "SELECT * FROM VENDAS"
query_vendedor = "SELECT * FROM VENDEDOR"


df_clientes = pd.read_sql(query_clientes, engine_origem)
df_categoria_clientes = pd.read_sql(query_categoria_clientes, engine_origem)
df_categoria_produto = pd.read_sql(query_categoria_produto, engine_origem)
df_fornecedores = pd.read_sql(query_fornecedores, engine_origem)
df_item_vendas = pd.read_sql(query_item_vendas, engine_origem)
df_localidade = pd.read_sql(query_localidade, engine_origem)
df_lojas = pd.read_sql(query_lojas, engine_origem)
df_produtos = pd.read_sql(query_produtos, engine_origem)
df_produto_fornecedor = pd.read_sql(query_produto_fornecedor, engine_origem)
df_promocoes = pd.read_sql(query_promocoes, engine_origem)
df_vendas = pd.read_sql(query_vendas, engine_origem)
df_vendedor = pd.read_sql(query_vendedor, engine_origem)

# 2. Transformação dos dados

### Dimensão clientes

In [516]:
# Idades que são nulas, menores que 1 e maiores que 100 recebem a média das idades como valor. Idade passa a ser inteiro.

idade_para_media = df_clientes['idade'].copy()

idade_para_media[idade_para_media < 1] = np.nan
idade_para_media[idade_para_media > 100] = np.nan
media_idades_validas = idade_para_media.mean()
media_idades_validas = round(media_idades_validas)

condicao_idade_invalida = (df_clientes['idade'] < 1) | \
                          (df_clientes['idade'] > 100) | \
                          (df_clientes['idade'].isnull())

df_clientes.loc[condicao_idade_invalida, 'idade'] = media_idades_validas
df_clientes['idade'] = df_clientes['idade'].astype(int)

# Corrigindo valores para gênero

mapeamento_genero = {
    'F': 'Feminino',
    'M': 'Masculino',
    'Feminino': 'Feminino',
    'Masculino': 'Masculino',
    'outro': 'Outro'
}

df_clientes['genero'] = df_clientes['genero'].replace(mapeamento_genero)
df_clientes['genero'] = df_clientes['genero'].fillna('Não informado')

# Adicionando valor de categoria ao cliente

df_clientes = pd.merge(
    df_clientes,
    df_categoria_clientes,
    on='id_categoria_cliente',
    how='left'
)
df_clientes['nome_categoria_cliente'] = df_clientes['nome_categoria_cliente'].fillna('Categoria não informada')


# Criando dimensão do cliente

dim_clientes = df_clientes[[
    'id_cliente',
    'nome_cliente',
    'idade',
    'genero',
    'nome_categoria_cliente'
]].rename(columns={
    'nome_cliente': 'nome',
    'nome_categoria_cliente': 'categoria'
})



### Dimensões de cidade, estado e região

In [517]:
# Padronização dos valores (cidade, estado e região)

df_localidade['cidade'] = df_localidade['cidade'].str.lower()
df_localidade['cidade'] = df_localidade['cidade'].str.strip()

df_localidade['regiao'] = df_localidade['regiao'].str.lower()
df_localidade['regiao'] = df_localidade['regiao'].str.strip()

df_localidade['estado'] = df_localidade['estado'].str.upper()
df_localidade['estado'] = df_localidade['estado'].str.strip()

# Adição de ID para cidade, estado e região

df_localidade['id_cidade'] = pd.factorize(df_localidade['cidade'])[0] + 1
df_localidade['id_regiao'] = pd.factorize(df_localidade['regiao'])[0] + 1
df_localidade['id_estado'] = pd.factorize(df_localidade['estado'])[0] + 1

# Criação de dimensões

dim_cidade = df_localidade[['cidade', 'id_cidade']].drop_duplicates(subset=['id_cidade'])
dim_estado = df_localidade[['estado', 'id_estado']].drop_duplicates(subset=['id_estado'])
dim_regiao = df_localidade[['regiao', 'id_regiao']].drop_duplicates(subset=['id_regiao'])


# Criação de registro para cidades, estados e regiões inválidas

registro_cidade_desconhecida = pd.DataFrame({
    'cidade': ['Cidade Desconhecida'],
    'id_cidade': [-1]
})
dim_cidade = pd.concat([dim_cidade, registro_cidade_desconhecida], ignore_index=True)

registro_estado_desconhecido = pd.DataFrame({
    'estado': ['Estado Desconhecido'],
    'id_estado': [-1]
})
dim_estado = pd.concat([dim_estado, registro_estado_desconhecido], ignore_index=True)

registro_regiao_desconhecida = pd.DataFrame({
    'regiao': ['Região Desconhecida'],
    'id_regiao': [-1]
})
dim_regiao = pd.concat([dim_regiao, registro_regiao_desconhecida], ignore_index=True)


### Dimensão Produto

In [518]:
# Adição das colunas de categoria_produto ao dataframe
df_produtos = pd.merge(
    df_produtos,
    df_categoria_produto,
    on='id_categoria_produto',
    how='left'
)

# Tratamento de valores None e vazio no nome do produto
nomes_produtos = {
    '': 'Nome não informado',
    None: 'Nome não informado',
}
df_produtos['nome_produto'] = df_produtos['nome_produto'].replace(nomes_produtos)


# Tratamento do valores None e NaN na categoria do produto
df_produtos['nome_categoria_produto'] = df_produtos['nome_categoria_produto'].fillna('Sem categoria')
dim_produto = df_produtos[['id_produto', 'nome_produto', 'nome_categoria_produto']].rename(columns={'nome_categoria_produto': 'categoria', 'nome_produto': 'nome'})

### Dimensão Fornecedores

In [519]:
# Usa a condição para localizar as linhas com país nulo e atribuir o valor "PAIS NÃO INFORMADO".

valor_substituto = "PAIS NÃO INFORMADO"

condicao_pais_nulo = df_fornecedores['pais_origem'].isnull()

df_fornecedores.loc[condicao_pais_nulo, 'pais_origem'] = valor_substituto

# Padronização dos nomes dos paises

df_fornecedores['pais_origem'] = df_fornecedores['pais_origem'].str.upper().str.strip()

mapeamento_paises = {
    'BR': 'BRASIL',
    'BRAZIL': 'BRASIL',
    'CN': 'CHINA',
    'USA': 'ESTADOS UNIDOS',
    'EUA': 'ESTADOS UNIDOS'
}

df_fornecedores['pais_origem'] = df_fornecedores['pais_origem'].replace(mapeamento_paises)

dim_fornecedores = df_fornecedores

dim_fornecedores = dim_fornecedores.rename(columns={
    'nome_fornecedor': 'nome',
    'pais_origem': 'pais'
})

### Dimensão Vendedor

In [520]:
# Remoção de prefixos nos nomes

prefixos_a_remover = [r'SR\.', r'SRA\.', r'DR\.', r'DRA\.', r'Srta\.']
padrao_regex = r'^(?:' + '|'.join(prefixos_a_remover) + r')\s*'

df_vendedor['nome_vendedor'] = df_vendedor['nome_vendedor'].str.replace(
    padrao_regex,
    '',
    regex=True,
    flags=re.IGNORECASE
).str.strip()

dim_vendedor = df_vendedor


dim_vendedor = dim_vendedor.rename(columns={
    'nome_vendedor': 'nome',
})

### Dimensão Promoção

In [521]:
# Preenche os valores nulos em tipo_desconto com "Não informado"

df_promocoes['tipo_desconto'] = df_promocoes['tipo_desconto'].fillna('NAO INFORMADO')

# Unifica os sinônimos e símbolos para um padrão

df_promocoes['tipo_desconto'] = df_promocoes['tipo_desconto'].str.upper().str.strip()

mapeamento_desconto = {
    'FIXO': 'VALOR FIXO',
    '%': 'PERCENTUAL'
}
df_promocoes['tipo_desconto'] = df_promocoes['tipo_desconto'].replace(mapeamento_desconto)

# Corrigir inconsistencias na coluna "data_inicio".

df_promocoes['data_inicio'] = df_promocoes['data_inicio'].replace(['N/A', 'Data Inválida', ''], np.nan)

df_promocoes['data_inicio'] = pd.to_datetime(df_promocoes['data_inicio'], errors='coerce')

# Corrigir inconsistencias na coluna "data_fim".

df_promocoes['data_fim'] = df_promocoes['data_fim'].replace(['Data Inválida', ''], np.nan)

df_promocoes['data_fim'] = pd.to_datetime(df_promocoes['data_fim'], errors='coerce')

dim_promocao = df_promocoes


# Adição de registro de promoção desconhecida
registro_promocao_desconhecida = pd.DataFrame({
    'id_promocao': [-1],
    'nome_promocao': ['Promoção Desconhecida'],
    'tipo_desconto': ['NÃO APLICÁVEL'],
    'data_inicio': [pd.NaT],
    'data_fim': [pd.NaT]
})

dim_promocao = pd.concat([dim_promocao, registro_promocao_desconhecida], ignore_index=True)

dim_promocao = dim_promocao.rename(columns={
    'nome_promocao': 'nome',
    'tipo_desconto': 'tipo',
})


### Dimensão Loja

In [522]:
# Resolvendo as inconsistencias de cidade

df_lojas['cidade'] = df_lojas['cidade'].replace('', np.nan)

df_lojas['cidade'] = df_lojas['cidade'].fillna('CIDADE NÃO INFORMADA')

# Remove prefixos dos nomes dos gerentes

prefixos_a_remover = [r'SR\.', r'SRA\.', r'DR\.', r'DRA\.', r'Srta\.']
padrao_regex = r'^(?:' + '|'.join(prefixos_a_remover) + r')\s*'
df_lojas['gerente_loja'] = df_lojas['gerente_loja'].str.replace(
    padrao_regex,
    '',
    regex=True,
    flags=re.IGNORECASE
).str.strip()

dim_lojas = df_lojas.rename(columns={'nome_loja': 'nome', 'gerente_loja': 'gerente'})


In [523]:
dataframes = {
        "dim_clientes": dim_clientes,
        "dim_fornecedores": dim_fornecedores,
        "dim_lojas": dim_lojas,
        "dim_produtos": dim_produto,
        "dim_promocao": dim_promocao,
        "dim_vendedor": dim_vendedor,
}

for df_name, df in dataframes.items():
        print(f"--- DataFrame: {df_name} ---")
        print("\nColunas:")
        print(df.columns.tolist())
        print("\n" + "="*50 + "\n")

--- DataFrame: dim_clientes ---

Colunas:
['id_cliente', 'nome', 'idade', 'genero', 'categoria']


--- DataFrame: dim_fornecedores ---

Colunas:
['id_fornecedor', 'nome', 'pais']


--- DataFrame: dim_lojas ---

Colunas:
['id_loja', 'nome', 'gerente', 'cidade', 'estado']


--- DataFrame: dim_produtos ---

Colunas:
['id_produto', 'nome', 'categoria']


--- DataFrame: dim_promocao ---

Colunas:
['id_promocao', 'nome', 'tipo', 'data_inicio', 'data_fim']


--- DataFrame: dim_vendedor ---

Colunas:
['id_vendedor', 'nome']




### Criando dados para entidades inválidas nas tabelas

In [524]:
# 1. dim_clientes
registro_cliente_desconhecido = pd.DataFrame({
    'id_cliente': [-1],
    'nome': ['Cliente Desconhecido'],
    'idade': [np.nan], # Idade pode ser NaN
    'genero': ['Não Informado'],
    'categoria': ['Não Informada']
})
dim_clientes = pd.concat([dim_clientes, registro_cliente_desconhecido], ignore_index=True)
dim_clientes['idade'] = dim_clientes['idade'].astype('Int64') # Ajustar tipo para nullable int


# 2. dim_fornecedores
registro_fornecedor_desconhecido = pd.DataFrame({
    'id_fornecedor': [-1],
    'nome': ['Fornecedor Desconhecido'],
    'pais': ['Não Informado']
})
dim_fornecedores = pd.concat([dim_fornecedores, registro_fornecedor_desconhecido], ignore_index=True)


# 3. dim_lojas
registro_loja_desconhecida = pd.DataFrame({
    'id_loja': [-1],
    'nome': ['Loja Desconhecida'],
    'gerente': ['Não Informado'],
    'cidade': ['Não Informada'],
    'estado': ['NI'] # NI para Não Informado
})
dim_lojas = pd.concat([dim_lojas, registro_loja_desconhecida], ignore_index=True)


# 4. dim_produtos
registro_produto_desconhecido = pd.DataFrame({
    'id_produto': [-1],
    'nome': ['Produto Desconhecido'],
    'categoria': ['Não Informada']
})
dim_produto = pd.concat([dim_produto, registro_produto_desconhecido], ignore_index=True)


# 5. dim_vendedor
registro_vendedor_desconhecido = pd.DataFrame({
    'id_vendedor': [-1],
    'nome': ['Vendedor Desconhecido']
})
dim_vendedor = pd.concat([dim_vendedor, registro_vendedor_desconhecido], ignore_index=True)

### Tabela fatos_vendas

In [525]:
# União de vendas com itens_venda
fato_vendas = pd.merge(
    df_vendas,
    df_item_vendas,
    on='id_venda',
    how='inner',
)


# Tratamento da coluna de data de vendas
fato_vendas['data_venda'] = fato_vendas['data_venda'].replace(['N/A', 'Data Inválida', '', None, 'None'], np.nan)
fato_vendas['data_venda'] = pd.to_datetime(fato_vendas['data_venda'], format='%Y-%m-%d', errors='coerce')


# Tratamento da coluna de quantidade de vendas
condicao_quantidade_produto = (fato_vendas['qtd_vendida'] < 1)
fato_vendas.loc[condicao_quantidade_produto, 'qtd_vendida'] = None
fato_vendas['qtd_vendida'] = fato_vendas['qtd_vendida'].astype('Int64')


# Criação das colunas de localidade do cliente
df_localidades_cliente = pd.merge(
    df_clientes,
    df_localidade,
    on='id_localidade',
    how='inner',
)

df_localidades_cliente = df_localidades_cliente.rename(columns={
    'id_cidade': 'id_cidade_cliente',
    'id_estado': 'id_estado_cliente',
    'id_regiao': 'id_regiao_cliente',
})[['id_cliente', 'id_cidade_cliente', 'id_estado_cliente', 'id_regiao_cliente']]


# Adição das colunas de id_cidade, id_estado, id_regiao a fato_vendas
fato_vendas = pd.merge(
    fato_vendas,
    df_localidades_cliente,
    on='id_cliente',
    how='left',
)

# Adição de id inválido para id's ausentes em cidade, estado e regiao
fato_vendas['id_cidade_cliente'] = fato_vendas['id_cidade_cliente'].fillna(-1).astype('Int64')
fato_vendas['id_estado_cliente'] = fato_vendas['id_estado_cliente'].fillna(-1).astype('Int64')
fato_vendas['id_regiao_cliente'] = fato_vendas['id_regiao_cliente'].fillna(-1).astype('Int64')


# Adição de id inválido para id's ausentes de promoção
fato_vendas['id_promocao_aplicada'] = fato_vendas['id_promocao_aplicada'].fillna(-1)
fato_vendas.loc[fato_vendas['id_promocao_aplicada'] == 0, 'id_promocao_aplicada'] = -1
fato_vendas['id_promocao_aplicada'] = fato_vendas['id_promocao_aplicada'].astype('Int64')

# Remoção de coluna de valor total
fato_vendas = fato_vendas.drop(['valor_total'], axis='columns')

fato_vendas['id_pedido'] = fato_vendas.index + 1

fato_vendas = fato_vendas.rename(columns={
    'qtd_vendida': 'quantidade_produtos',
    'id_promocao_aplicada': 'id_promocao'
})


### Adição de Id's inválidos ao dataframe de fato_vendas

In [526]:
# --- Função Auxiliar para Lookup e Tratamento de IDs ---
def map_dimension_id(
    df_fact: pd.DataFrame,
    fact_id_col: str,
    dim_df: pd.DataFrame,
    dim_pk_col: str,
    unknown_sk_value: int = -1
) -> pd.DataFrame:
    valid_dim_ids = set(dim_df[dim_df[dim_pk_col] != unknown_sk_value][dim_pk_col].unique())
    condicao_invalida_ou_nao_encontrada = (df_fact[fact_id_col].isna()) | \
                                         (~df_fact[fact_id_col].isin(valid_dim_ids))
    df_fact.loc[condicao_invalida_ou_nao_encontrada, fact_id_col] = unknown_sk_value
    df_fact[fact_id_col] = df_fact[fact_id_col].astype('Int64')

    return df_fact

# --- Aplicar a função para cada dimensão ---

# 1. Dimensão Loja
fato_vendas = map_dimension_id(fato_vendas, 'id_loja', dim_lojas, 'id_loja')

# 2. Dimensão Promoção
fato_vendas = map_dimension_id(fato_vendas, 'id_promocao', dim_promocao, 'id_promocao')

# 3. Dimensão Produto
fato_vendas = map_dimension_id(fato_vendas, 'id_produto', dim_produto, 'id_produto')

# 4. Dimensão Cliente
fato_vendas = map_dimension_id(fato_vendas, 'id_cliente', dim_clientes, 'id_cliente')

# 5. Dimensão Vendedor
fato_vendas = map_dimension_id(fato_vendas, 'id_vendedor', dim_vendedor, 'id_vendedor')

# 6. Dimensão Cidade Cliente
fato_vendas = map_dimension_id(fato_vendas, 'id_cidade_cliente', dim_cidade, 'id_cidade')

# 7. Dimensão Estado Cliente
fato_vendas = map_dimension_id(fato_vendas, 'id_estado_cliente', dim_estado, 'id_estado')

# 8. Dimensão Região Cliente
fato_vendas = map_dimension_id(fato_vendas, 'id_regiao_cliente', dim_regiao, 'id_regiao')

### Tabela fato_fornecimento

In [528]:
# Tratamento de valor compra unitário de produto

condicao_preco_compra_invalido = (df_produto_fornecedor['custo_compra_unitario'] < 1)
df_produto_fornecedor.loc[condicao_preco_compra_invalido, 'custo_compra_unitario'] = None
df_produto_fornecedor['custo_compra_unitario'] = df_produto_fornecedor['custo_compra_unitario'].astype(float).round(2)

fato_fornecimento = df_produto_fornecedor

# 3. Carregamento dos dados

### Carregamento das dimensões

In [529]:
def carregar_dimensao(df_dimensao: pd.DataFrame, dimensao: str):
    df_dimensao.to_sql(
            name=dimensao,
            con=engine_dw,
            if_exists='append',
            index=False
    )

carregar_dimensao(dim_lojas, 'dm_loja')
carregar_dimensao(dim_clientes, 'dm_cliente')
carregar_dimensao(dim_promocao, 'dm_promocao')
carregar_dimensao(dim_cidade, 'dm_cidade')
carregar_dimensao(dim_vendedor, 'dm_vendedor')
carregar_dimensao(dim_estado, 'dm_estado')
carregar_dimensao(dim_fornecedores, 'dm_fornecedor')
carregar_dimensao(dim_regiao, 'dm_regiao')
carregar_dimensao(dim_produto, 'dm_produto')


### Carregamento das tabelas Fato

In [530]:
def carregar_fato(df_fato: pd.DataFrame, fato: str):
    df_fato.to_sql(
            name=fato,
            con=engine_dw,
            if_exists='append',
            index=False
    )

carregar_fato(fato_vendas, 'fato_vendas')
carregar_fato(fato_fornecimento, 'fato_fornecimento')

# 4. Ver tabelas geradas

In [531]:
dim_clientes

,id_cliente,nome,idade,genero,categoria
0,1,Ana Sophia Silveira,48,Feminino,Categoria Cliente D
1,2,Dr. Davi Miguel Almeida,63,Não informado,Categoria Cliente E
2,3,Sr. Luiz Otávio Farias,28,Masculino,Categoria Cliente B
3,4,Ana Beatriz Monteiro,58,Feminino,Categoria não informada
4,5,Lorenzo Duarte,23,Feminino,Categoria não informada
...,...,...,...,...,...
7996,7997,Mariah Souza,51,Feminino,Categoria Cliente C
7997,7998,Antônio Peixoto,56,Masculino,Categoria Cliente F
7998,7999,Isis Rocha,72,Feminino,Categoria não informada
7999,8000,Bárbara Garcia,52,Feminino,Categoria não informada


In [532]:
dim_lojas

,id_loja,nome,gerente,cidade,estado
0,1,Loja Brito do Campo,João Miguel Duarte,Sousa,PA
1,2,Loja Guerra Verde,Davi Miguel Pastor,Oliveira,RO
2,3,Loja Souza,José Pedro Duarte,Correia,MT
3,4,Loja Cunha,Liz Melo,Carvalho do Norte,PI
4,5,Loja Machado,Maria Castro,Fonseca,PI
5,6,Loja Vasconcelos do Norte,Aylla Viana,Fonseca,PA
6,7,Loja Monteiro do Amparo,Igor da Rosa,CIDADE NÃO INFORMADA,BA
7,8,Loja das Neves,Marcela Vasconcelos,Rios,AC
8,9,Loja Fernandes,Bianca Farias,Porto,MG
9,10,Loja Caldeira,Caio Silva,Dias de Nascimento,AC


In [533]:
dim_promocao

,id_promocao,nome,tipo,data_inicio,data_fim
0,1,Promoção Soluta,VALOR FIXO,2024-12-30,NaT
1,2,Promoção Veniam,NAO INFORMADO,2025-01-29,2025-03-15
2,3,Promoção Assumenda,VALOR FIXO,2024-06-05,2024-07-23
3,4,Promoção Dolor,PERCENTUAL,2022-07-18,2022-08-02
4,5,Promoção Doloribus,NAO INFORMADO,2024-06-08,2024-07-31
5,6,Promoção Qui,NAO INFORMADO,NaT,2025-06-11
6,7,Promoção Eveniet,NAO INFORMADO,2024-12-30,2025-01-23
7,8,Promoção Eaque,VALOR FIXO,2022-10-12,2022-11-02
8,9,Promoção Pariatur,NAO INFORMADO,NaT,NaT
9,10,Promoção Possimus,NAO INFORMADO,NaT,2024-10-20


In [534]:
dim_vendedor

,id_vendedor,nome
0,1,Lívia da Rocha
1,2,Levi Borges
2,3,Ana Laura Fernandes
3,4,Luiz Fernando Vargas
4,5,Davi Lucas Pereira
...,...,...
496,497,Antony Nascimento
497,498,Bryan Ferreira
498,499,Pedro Henrique Santos
499,500,Heitor Costa


In [535]:
dim_fornecedores

,id_fornecedor,nome,pais
0,1,Araújo,BRASIL
1,2,Pastor,BRASIL
2,3,Novais,ESTADOS UNIDOS
3,4,Jesus,CHINA
4,5,Souza,BRASIL
...,...,...,...
196,197,Freitas S/A,CHINA
197,198,Peixoto Ltda.,CHINA
198,199,Oliveira,PAIS NÃO INFORMADO
199,200,Santos Souza Ltda.,CHINA


In [536]:
dim_produto

,id_produto,nome,categoria
0,1,Nome não informado,Sem categoria
1,2,Nome não informado,Categoria Prod M
2,3,Nome não informado,Categoria Prod J
3,4,Nome não informado,Categoria Prod K
4,5,Produto F5,Sem categoria
...,...,...,...
1996,1997,Nome não informado,Sem categoria
1997,1998,Produto W1998,Categoria Prod O
1998,1999,Nome não informado,Sem categoria
1999,2000,Nome não informado,Sem categoria


In [537]:
dim_regiao

,regiao,id_regiao
0,rio de janeiro,1
1,piauí,2
2,roraima,3
3,maranhão,4
4,amapá,5
5,espírito santo,6
6,tocantins,7
7,distrito federal,8
8,paraíba,9
9,goiás,10


In [538]:
dim_cidade

,cidade,id_cidade
0,albuquerque,1
1,ramos de sousa,2
2,mendonça da mata,3
3,da cruz alegre,4
4,fonseca,5
...,...,...
85,cassiano de minas,86
86,da cunha de minas,87
87,da rocha dos dourados,88
88,abreu de aragão,89


In [539]:
dim_estado

,estado,id_estado
0,AP,1
1,MG,2
2,RO,3
3,MS,4
4,TO,5
5,PB,6
6,SC,7
7,PI,8
8,RR,9
9,PA,10
